In [1]:
import warnings
warnings.filterwarnings('ignore') #filters out annoying warnings in PCA output

In [2]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from statsmodels.discrete.discrete_model import Logit
from scipy.stats import ttest_ind, f_oneway
from utility_functions import load_file, pickle_file, starting_run, finished_run
from analysis_variables import logreg_targets, de_col_keys, de_col_values, outcome_cols

In [3]:
filtered_data = load_file("summary_costs_enhanced.pickle")
category_status = load_file("category_status_filtered.pickle")
pca = PCA(n_components=10)
enc = OneHotEncoder(sparse=False)
scaler = StandardScaler()
mScaler = MinMaxScaler()
if not os.path.isdir(f"../tables/logreg"):
    os.mkdir(f"../tables/logreg")

In [4]:
demographic_cols = ['marital_status', 'initial_discharge_quarter', 'gender', 'race', 'payer']
def encode_dataset(dataset):
    encoded_dataset = pd.DataFrame(
        enc.fit_transform(dataset), dataset.index
    )
    encoded_dataset.columns = enc.get_feature_names(dataset.columns)
    return encoded_dataset
def preprocess_dataset(dataset):
    #encode dataset demographics
    dem_dataset = dataset.loc[:, demographic_cols].dropna()
    encoded_dataset = encode_dataset(dem_dataset).join(dataset["age"], how='inner').join(category_status, how="inner")
    #scale columns
    scaled_data = pd.DataFrame(
        scaler.fit_transform(encoded_dataset),
        index = encoded_dataset.index,
        columns = encoded_dataset.columns
    )
    pca_data = pd.DataFrame(
        mScaler.fit_transform(encoded_dataset),
        index = encoded_dataset.index,
        columns = encoded_dataset.columns
    )
    return scaled_data, pca_data

In [5]:
def run_logreg(dataset, target):
    return Logit(target, dataset).fit_regularized(maxiter=1000, disp=False).summary2().tables[1].sort_values(['P>|z|', 'Coef.'])

In [6]:
def run_PCA(dataset):
    print(dataset.shape)
    fitted_model = pca.fit(dataset)
    return pd.DataFrame(scaler.fit_transform(fitted_model.transform(dataset)), index=dataset.index), fitted_model.explained_variance_ratio_, pd.DataFrame(fitted_model.components_.T, index=dataset.columns)

In [7]:
def ttest(target):
    dataset = filtered_data.join(target.rename("target"), how="inner")
    stats = pd.DataFrame({
        'Metric': outcome_cols,
        'Mean True': [dataset.loc[dataset["target"] == True][col].mean() for col in outcome_cols],
        'Mean False': [dataset.loc[dataset["target"] == False][col].mean() for col in outcome_cols],
        'P value': [
            ttest_ind(
                dataset.loc[dataset["target"] == True][col],
                dataset.loc[dataset["target"] == False][col],
                equal_var=False
            ).pvalue*2 for col in outcome_cols
        ]
    })
    print(stats)
    return stats

In [8]:
datasets = {
    "Full": filtered_data,
#     "Inflamed": filtered_data.loc[category_status["has biliary colic with inflammation"].eq(1)],
#     "Uninflamed": filtered_data.loc[category_status["has biliary colic with inflammation"].eq(0)]
}
explained_variance = {}
ttest_results = []
for name, dataset in datasets.items():
    starting_run(name)
    scaled_data, pca_data = preprocess_dataset(dataset)
    pca_dataset, component_importance, component_eigenvalues = run_PCA(pca_data)
    dataset = dataset.loc[scaled_data.index]
    explained_variance[name] = component_importance
#     if name != "Full":
#         del scaled_data["has biliary colic with inflammation"]
    for target_name, target_function in logreg_targets.items():
        starting_run(target_name)
        target_data = target_function(dataset)
        print(target_data.value_counts())
        run_logreg(scaled_data.loc[target_data.index], target_data).to_csv(f"../tables/logreg/{target_name} Feature Scores.csv")
        run_logreg(pca_dataset.loc[target_data.index], target_data).to_csv(f"../tables/logreg/{target_name} PCA Component Scores.csv")
        ttest_results.append(ttest(target_data))
        print()
        print()
    component_eigenvalues.to_csv(f"../tables/logreg/PCA eigenvalues.csv")
#     pickle_file(f"{name}_logreg_targets.pickle", targets)
pd.DataFrame(explained_variance).to_csv(f"../tables/logreg/PCA explained variance.csv")

Starting Full 11:49:06.049733
(5547, 33)
Starting Immediate Cholecystectomy vs Others for Complicated Colic 11:49:06.150673
True     3528
False    1250
Name: Cholecystectomy Type, dtype: int64
                   Metric     Mean True   Mean False       P value
0                    Cost  10795.296551  9093.553402  2.665395e-09
1  Inpatient Readmissions      0.011054     0.140000  4.647595e-32
2         ED Readmissions      0.011905     0.088000  2.825065e-18
3                    Died      0.000000     0.000800  6.350084e-01


Starting Discharge vs Admission for Uncomplicated Colic 11:49:06.319733
False    495
True     274
Name: Admission Status, dtype: int64
                   Metric   Mean True   Mean False        P value
0                    Cost  969.160782  9597.609460  1.440181e-111
1  Inpatient Readmissions    0.007299     0.028283   6.971070e-02
2         ED Readmissions    0.018248     0.014141   1.435143e+00
3                    Died    0.000000     0.000000            NaN


Sta